In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [13]:
def auto_Scrapper_Class(html_tag,course_case,tag_class, div_class=None):
    """
    The function auto_Scrapper_Class is used to get three parameters that is the tag,what to scrap and get the content scrapped and class it belongs. 
    """
    for i in range(1,84): # adjust as needed, according to current coursera website, there are 83 pages for all courses
        url = "https://www.coursera.org/courses?page=" +str(i)
        
        #Use below url to gain more customization on the result with different query
        #url = "https://www.coursera.org/search?query=data%20science&page=" +str(i)
        
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        if div_class:
            elements = soup.find_all('div',  class_ = div_class)

            if (len(elements)) != 12:  
                for j in range(0,12):    # There are 12 courses per page
                    course_case.append(None)
                continue
            for name in elements:
                x = name.get_text()
                if x:
                    course_case.append(x)
                else:
                    course_case.append(None)

        else:
            element = soup.find_all(html_tag,  class_ = tag_class)
            if (len(element)) != 12:
                for j in range(0,12):
                    course_case.append(None)
                continue

            for name in element:
                x = name.get_text()
                if x:
                    course_case.append(x)
                else:
                    course_case.append("")

In [15]:
course_title = []
course_organization = []
course_rating = []
course_difficulty = []
course_review_counts = []
course_skills = []

In [17]:
#scrap the other information as per coursera's website html

#scrap the course title
auto_Scrapper_Class('h3',course_title, tag_class='cds-CommonCard-title css-6ecy9b')
#scrap the organization
auto_Scrapper_Class('p',course_organization,'cds-ProductCard-partnerNames css-vac8rf')
#scrap course rating
auto_Scrapper_Class('span',course_rating,'css-6ecy9b')
#scrap course difficulty (Metadata)
auto_Scrapper_Class('p',course_difficulty,'cds-119 cds-Typography-base css-dmxkm1 cds-121', 'cds-CommonCard-metadata')
#scrap review counts
auto_Scrapper_Class('div',course_review_counts,'css-vac8rf')
#scrap course skills
auto_Scrapper_Class('p',course_skills,'cds-119 cds-Typography-base css-dmxkm1 cds-121', 'cds-CommonCard-bodyContent' )

In [19]:
# Clean skills column and create df

data = {
    'Title': course_title,
    'Organization': course_organization,
    'Skills': course_skills,
    'Ratings':course_rating,
    'Review counts':course_review_counts,
    'Metadata': course_difficulty    
}
df = pd.DataFrame(data)
df['Skills'] = df['Skills'].str.replace("Skills you'll gain:", '', regex=False)
df

,Title,Organization,Skills,Ratings,Review counts,Metadata
0,Google Data Analytics,Google,"Data Storytelling, Rmarkdown, Data Visualizat...",4.8,164K reviews,Beginner · Professional Certificate · 3 - 6 Mo...
1,Google Cybersecurity,Google,"Threat Modeling, Open Web Application Securit...",4.8,48K reviews,Beginner · Professional Certificate · 3 - 6 Mo...
2,Google Project Management:,Google,"Quality Management, Project Scoping, Project ...",4.8,125K reviews,Beginner · Professional Certificate · 3 - 6 Mo...
3,Google Digital Marketing & E-commerce,Google,"Media Planning, Search Engine Marketing, Emai...",4.8,37K reviews,Beginner · Professional Certificate · 3 - 6 Mo...
4,Google IT Support,Google,"Computer Networking, Network Troubleshooting,...",4.8,200K reviews,Beginner · Professional Certificate · 3 - 6 Mo...
...,...,...,...,...,...,...
991,"Data Modeling, Transformation, and Serving",DeepLearning.AI,"Data Modeling, Data Transformation, Data Proc...",4.6,59 reviews,Intermediate · Course · 1 - 4 Weeks
992,Social Media Marketing,Northwestern University,"Social Media Marketing, Integrated Marketing ...",4.6,5.5K reviews,Beginner · Specialization · 3 - 6 Months
993,AI and Public Health,DeepLearning.AI,"Data Analysis, Jupyter, Exploratory Data Anal...",4.8,216 reviews,Beginner · Course · 1 - 4 Weeks
994,Understanding the Enterprise Systems Environment,SAP,"Digital Transformation, Cloud Computing Archi...",4.7,195 reviews,Beginner · Course · 1 - 3 Months


In [21]:
#create csv with df
df.to_csv("coursera_course_dataset.csv")

In [23]:
#Separate skills list to create distinct skills from coursera df

skills_column = df['Skills']

skills_column = [str(skill) if skill is not None else '' for skill in skills_column]

# Concatenate all skills into a single string
all_skills_text = ', '.join(skills_column)

# Split the string into a list of skills
all_skills_list = [skill.strip() for skill in all_skills_text.split(',')]

# Get unique skills
distinct_skills = list(set(all_skills_list))

distinct_skills_df = pd.DataFrame({'Distinct Skills': distinct_skills,'source':"coursera"})
len(distinct_skills)

1700

In [25]:
distinct_skills_df

,Distinct Skills,source
0,,coursera
1,General Medical Tests and Procedures,coursera
2,Interaction Design,coursera
3,Mortgage Loans,coursera
4,Cooking,coursera
...,...,...
1695,Technology Strategies,coursera
1696,Data Wrangling,coursera
1697,Strategic Thinking,coursera
1698,Recruitment Strategies,coursera


In [27]:
# create csv with distinct_skills_df
distinct_skills_df.to_csv("skills_coursera.csv")
